In [1]:
path_goal="/mnt/ceph/storage/data-in-progress/data-research/arguana/args/args-near-duplicates-mapper/anserini-indices/version-1.0-cleaned.jsonl"
path_source="/mnt/ceph/storage/data-in-progress/data-research/arguana/args/args-near-duplicates-mapper/anserini-indices/version-2020-04-01.jsonl"
path_similarities="/mnt/ceph/storage/.snap/daily_20210804/data-in-progress/data-research/arguana/args/args-near-duplicates-mapper/s3-similarities/s3-scores-for-index-1.0-and-cleaned-2020-04-01.jsonl"
path_judgements="/mnt/ceph/storage/data-in-progress/data-research/arguana/args/args-near-duplicates-mapper/beir/touche2020-task1-version-2020-04-01-corrected.qrels"
path_all_score_mappings="/mnt/ceph/storage/data-in-progress/data-research/arguana/args/args-near-duplicates-mapper/validation/all-id-mappings.csv"
path_all_url_mappings="/mnt/ceph/storage/data-in-progress/data-research/arguana/args/args-near-duplicates-mapper/validation/all-url-mappings.csv"
path_mappings="/mnt/ceph/storage/data-in-progress/data-research/arguana/args/args-near-duplicates-mapper/validation/mappings.csv"

In [2]:
import ijson
from trectools import TrecQrel, procedures
file_goal = open (path_goal,'r',encoding="utf-8")
file_source = open(path_source,'r',encoding="utf-8")
qrels = TrecQrel(path_judgements)
df_qrels=qrels.qrels_data
df_qrels.rename({'docid':'source-id'},inplace=True)

In [3]:
import pandas as pd
i = 0
for i,qrel in df_qrels.iterrows():
    file_similarities = open(path_similarities,'r',encoding="utf-8")
    file_similarities_2 = open(path_similarities,'r',encoding="utf-8")
    scores=ijson.items(file_similarities,'s3Score',multiple_values=True)
    pairs = ijson.items(file_similarities_2,'idPair',multiple_values=True)
    pairs_similarities=[]
    b=-1
    for pair in pairs:
        score = next(scores)
        left_id = pair['left']
        right_id = pair['right']
        if left_id == qrel['source-id']:
            pairs_similarities.append((left_id,right_id,score))
            b=1
            break
        if right_id == qrel['source-id']:
            pairs_similarities.append((right_id,left_id,score))
            b=1
            break
        if b==1:
            break
    file_similarities.close()
    file_similarities_2.close()
print(id_pairs)
ids,goal_ids,scores=zip(*id_pairs)
df_all_score_mappings=pd.DataFrame({'source-id':ids,'goal-id':goal_ids,'score':scores})
df_all_score_mappings.to_csv(path_all_score_mappings,sep=",",encoding="utf-8")


KeyboardInterrupt: 

In [ ]:

    print(qrel['docid'])

In [ ]:
def entry2argument(entry):
    conclusion = entry['conclusion'].lower().replace("\t"," ")
    premise = entry['premises'][0]['text'].lower().replace("\t"," ")
    argument_id = entry['id']
    Argument = namedtuple('Argument', 'id conclusion premise url')
    argument_text = conclusion + " " + premise
    argument_text = clean(argument_text)
    if 'sourceUrl' in entry['context']:
        url = entry['context']['sourceUrl']
    else:
        url=None
    return Argument(argument_id,conclusion,premise,hash,url)


In [ ]:
import pandas as pd
def get_id_url_map(json_file,id_column):
    arguments= ijson.items(json_file,'arguments.item')
    parsed_arguments= [entry2argument(argument) for argument in arguments]
    ids=[parsed_argument.id for parsed_argument in all_parsed_arguments]
    urls=[parsed_argument.url for parsed_argument in all_parsed_arguments]
    df=pd.DataFrame({id_column:ids,'url':urls})
    return df

df_source_url=get_id_url_map(file_source,'source-id')
df_source_url=df_source_url.merge(df_qrels,on='source-id')
df_goal_url=get_id_url_map(file_goal,'goal-id')
df_url_mappings=df_source_url.merge(df_goal_url,on='url')
df_url_mappings.to_csv(path_all_url_mappings,sep=",",encoding="utf-8")

In [ ]:
df_all_score_mappings=pd.read_csv(path_all_score_mappings,sep=",",encoding="utf-8")
df_mappings=df_all_score_mappings.merge(df_url_mappings,on=['source-id','goal-id'])
df_mappings.to_csv(path_mappings,sep=",",encoding="utf-8")